In [1]:
import os
import yaml

import numpy as np

from probability_models.probability_model import ProbabilityModel
from probability_models.mixture_model import MixtureModel
import os
from atari_benchmark import *
from slgep_lib import *
from mfea_lib import *
from utils import Evaluator
from tools import Tools

from joblib import Parallel, delayed
from os import cpu_count
from tqdm import trange
np.seterr(under='ignore')

{'divide': 'raise', 'over': 'raise', 'under': 'raise', 'invalid': 'raise'}

In [2]:
# AMTEA solving KP_wc_ak
reps = 30;
TrInt = 2;

trans = {}
trans['transfer'] = True;
trans['TrInt'] = TrInt; 

In [3]:
# Load configuration
config = yaml.load(open('config.yaml').read())

# Load benchmark
singletask_benchmark = yaml.load(open('atari_benchmark/singletask-benchmark.yaml').read())
data = singletask_benchmark['single-5']
config.update(data)
config = wrap_config(config)

transfer = trans['transfer']
if transfer:
    TrInt = trans['TrInt']
    all_models = Tools.load_from_file(os.path.join('problems', 'all_models'))

# Problem
taskset = Taskset(config)

# Model
cf = ChromosomeFactory(config)

# Simple parameter
K = taskset.K
config['K'] = K
N = config['pop_size'] * K
T = config['num_iter']
dims = config['dim']
mutation_rate = config['mutation_rate']

bestSol = np.zeros((reps, dims))
alpha = [None] * (reps)

# Initialization
population = cf.initialize()
skill_factor = np.array([i % K for i in range(2 * N)])
factorial_cost = np.full([2 * N, K], np.inf)
scalar_fitness = np.empty([2 * N])

for rep in range(reps):
    alpha_rep = []

    # For parallel evaluation
    print('[+] Initializing evaluators')
    evaluators = [Evaluator(config) for _ in range(2 * N)]

    # First evaluation (sequential)
    delayed_functions = []
    for i in range(2 * N):
        sf = skill_factor[i]
        delayed_functions.append(delayed(evaluators[i].evaluate)(population[i], sf))
    fitnesses = Parallel(n_jobs=cpu_count())(delayed_functions)
    for i in range(2 * N):
        sf = skill_factor[i]
        factorial_cost[i, sf] = fitnesses[i]
    scalar_fitness = calculate_scalar_fitness(factorial_cost)
    
    # Evolve
#     iterator = trange(T)
    iterator = trange(10)
    for t in iterator:
        
        # permute current population
        permutation_index = np.random.permutation(N)
        population[:N] = population[:N][permutation_index]
        skill_factor[:N] = skill_factor[:N][permutation_index]
        factorial_cost[:N] = factorial_cost[:N][permutation_index]
        factorial_cost[N:] = np.inf
        
        
        # As we consider all the population as parents, we don't samplt P^{s}
        if transfer and t % TrInt == 0:
            mmodel = MixtureModel(all_models)
            mmodel.createtable(population, True, 'mvarnorm', config)
            mmodel.EMstacking();  # Recombination of probability models
            mmodel.mutate();  # Mutation of stacked probability model
            population = mmodel.sample(2 * N)
            alpha_rep.append(mmodel.alpha)
            print('Transfer coefficient at generation ', str(t), ': ', str(mmodel.alpha))
        else:
            # select pair to crossover
            for i in range(0, N, 2):
                # extract parent
                p1 = population[i]
                sf1 = skill_factor[i]
                p2 = find_relative(population, skill_factor, sf1, N)
                # recombine parent
                c1, c2 = cf.one_point_crossover_adf(p1, p2)
                c1 = cf.uniform_mutate(c1, mutation_rate)
                c2 = cf.uniform_mutate(c2, mutation_rate)
                # save child
                population[N + i, :], population[N + i + 1, :] = c1[:], c2[:]
                skill_factor[N + i] = sf1
                skill_factor[N + i + 1] = sf1
            
#         # evaluation
#         delayed_functions = []
#         for i in range(2 * N):
#             sf = skill_factor[i]
#             delayed_functions.append(delayed(evaluators[i].evaluate)(population[i], sf))
#         fitnesses = Parallel(n_jobs=cpu_count())(delayed_functions)
        for i in range(2 * N):
            sf = skill_factor[i]
            factorial_cost[i, sf] = fitnesses[i]
        scalar_fitness = calculate_scalar_fitness(factorial_cost)

        # sort
        sort_index = np.argsort(scalar_fitness)[::-1]
        population = population[sort_index]
        skill_factor = skill_factor[sort_index]
        factorial_cost = factorial_cost[sort_index]
        scalar_fitness = scalar_fitness[sort_index]

        # optimization info
        message = {'algorithm': 'cea', 'instance': config['names']}
        results = get_optimization_results(t, population, factorial_cost, scalar_fitness, skill_factor, message)
        desc = 'gen:{} fitness:{} message:{} K:{}'.format(t, ' '.join(
            '{:0.2f}|{:0.2f}|{:0.2f}'.format(res.fun, res.mean, res.std) for res in results), message, K)
        iterator.set_description(desc)




/home/quanhm/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  
/home/quanhm/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """


[+] Initializing evaluators


gen:1 fitness:-2000.00|-2000.00|0.00 message:{'algorithm': 'cea', 'instance': ['Atlantis-ram-v4']} K:1:  10%|█         | 1/10 [00:00<00:02,  4.35it/s]

Transfer coefficient at generation  0 :  [-0.02073273 -0.00364325  1.02437598]


gen:3 fitness:-2000.00|-2000.00|0.00 message:{'algorithm': 'cea', 'instance': ['Atlantis-ram-v4']} K:1:  30%|███       | 3/10 [00:00<00:01,  4.93it/s]

...

gen:7 fitness:-2000.00|-2000.00|0.00 message:{'algorithm': 'cea', 'instance': ['Atlantis-ram-v4']} K:1:  70%|███████   | 7/10 [00:01<00:00,  5.76it/s]

Transfer coefficient at generation  6 :  [-0.01337025 -0.01075356  1.02412381]


gen:9 fitness:-2000.00|-2000.00|0.00 message:{'algorithm': 'cea', 'instance': ['Atlantis-ram-v4']} K:1: 100%|██████████| 10/10 [00:01<00:00,  7.30it/s]

Transfer coefficient at generation  8 :  [-1.93716362e-02  9.95799949e-04  1.01837584e+00]
[+] Initializing evaluators


TypeError: list indices must be integers or slices, not float

In [ ]:
np.seterr(under='ignore')

In [ ]:
from scipy.stats import multivariate_normal

In [ ]:
multivariate_normal.pdf([[10,2,30,4],[1,2,3,4],[1,2,3,4],[2,3,4,5]],[30,2,3,6],[0,0.01,1,1], allow_singular = True)

In [ ]:
multivariate_normal.pdf([[10],[1],[1],[2]],[30],0, allow_singular = True)